In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [6]:
peptides=pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\train_peptides.csv")
proteins=pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\train_proteins.csv")
clinical_data=pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\train_clinical_data.csv")
supp_clinical=pd.read_csv(r"C:\Users\jshri\OneDrive\Desktop\supplemental_clinical_data.csv") 

In [7]:
all(proteins[['visit_id', 'UniProt']].value_counts() == 1)
df_p = peptides.merge(proteins[['visit_id', 'UniProt', 'NPX']], on=['visit_id','UniProt'], how='left')
df_p.head()
#I've rewritten visit_id for supplemental clinical data since it seems that its visit_id was different from convention in other files
supp_clinical['visit_id'] = supp_clinical['patient_id'].astype(str) + "_"+ supp_clinical['visit_month'].astype(str)

#Here we combine both main and supplemental clinical data into a single dataframe
df_cd = pd.concat([clinical_data, supp_clinical], ignore_index=True)
display(df_cd.info())
df_cd.melt(id_vars=['visit_id', 'patient_id', 'visit_month', 'upd23b_clinical_state_on_medication'], 
                   var_name='updrs', value_name='rating')
df_all = df_p.merge(df_cd[['visit_id','updrs_1','updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication']], on=['visit_id'], how='left')
df_all.info()
df_all['Peptide'].str.extract(r"(.\(.*?\))", expand=False).value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4838 entries, 0 to 4837
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             4838 non-null   object 
 1   patient_id                           4838 non-null   int64  
 2   visit_month                          4838 non-null   int64  
 3   updrs_1                              4624 non-null   float64
 4   updrs_2                              4622 non-null   float64
 5   updrs_3                              4808 non-null   float64
 6   updrs_4                              2872 non-null   float64
 7   upd23b_clinical_state_on_medication  2410 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 302.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981834 entries, 0 to 981833
Data columns (total 12 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   visit_id                             981834 non-null  object 
 1   visit_month                          981834 non-null  int64  
 2   patient_id                           981834 non-null  int64  
 3   UniProt                              981834 non-null  object 
 4   Peptide                              981834 non-null  object 
 5   PeptideAbundance                     981834 non-null  float64
 6   NPX                                  981834 non-null  float64
 7   updrs_1                              941744 non-null  float64
 8   updrs_2                              941744 non-null  float64
 9   updrs_3                              932624 non-null  float64
 10  updrs_4                              495530 non-null  float64
 11  upd23b_clinic

C(UniMod_4)     302757
M(UniMod_35)     25932
Name: Peptide, dtype: int64

In [8]:
import pandas as pd
df = df_all
# Create a new column that checks whether all UPDRS values are 0
df['all_UPDRS_0'] = ((df['updrs_1'] == 0) & (df['updrs_2'] == 0) &
                     (df['updrs_3'] == 0) & (df['updrs_4'] == 0))

# Group the control patients based on whether all UPDRS values are 0
control_groups = df[
                    (df['all_UPDRS_0'] == True)]

# Print the resulting groups
control_groups.head(40)
num_patients = control_groups['visit_month'].nunique()
print(num_patients)

2


In [9]:
df_all = df_all.drop(['updrs_4', 'upd23b_clinical_state_on_medication'], axis=1)

In [10]:
updrs_cutoffs = {'updrs_1': 1.5, 'updrs_2': 5, 'updrs_3': 13}

In [11]:
# Create a new column indicating whether an individual is less likely to have Parkinson's disease
df['no_pd'] = (df_all['updrs_1'] <= updrs_cutoffs['updrs_1']) & (df_all['updrs_2'] <= updrs_cutoffs['updrs_2']) &  (df_all['updrs_3'] <= updrs_cutoffs['updrs_3'])

In [12]:
# Separate no_pd == True (control) and no_pd == False (PD) into different dataframes
control_df = df[df['no_pd'] == True]
pd_df = df[df['no_pd'] == False]

# Drop updrs_4 and all_UPDRS_0 columns
control_df = control_df.drop(['updrs_4', 'all_UPDRS_0'], axis=1)
pd_df = pd_df.drop(['updrs_4', 'all_UPDRS_0'], axis=1)

In [13]:
control_df = control_df.drop(['visit_id', 'visit_month', 'UniProt', 'NPX','updrs_1', 'updrs_2', 'updrs_3', 'upd23b_clinical_state_on_medication' ,'no_pd'  ], axis=1) 

In [14]:
pd_df = pd_df.drop(['visit_id', 'visit_month', 'UniProt', 'NPX','updrs_1', 'updrs_2', 'updrs_3', 'upd23b_clinical_state_on_medication' ,'no_pd'  ], axis=1) 

In [15]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

# Separate no_pd == True (control) and no_pd == False (PD) into different dataframes
control_df = df[df['no_pd'] == True]
pd_df = df[df['no_pd'] == False]
control_df.fillna(0, inplace=True)
pd_df.fillna(0, inplace=True)

# Drop updrs_4 and all_UPDRS_0 columns
control_df = control_df.drop(['updrs_4', 'all_UPDRS_0'], axis=1)
pd_df = pd_df.drop(['updrs_4', 'all_UPDRS_0'], axis=1) 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    pd.concat([control_df, pd_df])[['PeptideAbundance']],
    pd.concat([control_df, pd_df])['no_pd'].astype(int),
    test_size=0.2, 
    random_state=7)

model = XGBClassifier(scale_pos_weight=1,
                      learning_rate=0.2,  
                      colsample_bytree = 0.9,
                      subsample = 0.3,
                      objective='reg:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=6, 
                      gamma=1)
model.fit(X_train,y_train)

# Test the model on the testing set
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

C:\Users\jshri\AppData\Local\Temp\ipykernel_13416\1584264029.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_df.fillna(0, inplace=True)
C:\Users\jshri\AppData\Local\Temp\ipykernel_13416\1584264029.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_df.fillna(0, inplace=True)


Accuracy: 0.8861315801534881
